<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Editado el 2018-1 y 2018-2 por Equipo Docente IIC2233.</font>
</p>

# Objetivos de la Unidad

- Conocer cómo se realizan las tareas en un sistema multitarea
- Conocer los conceptos de proceso y de _thread_
- Conocer y usar las bibliotecas de clases que provee Python para el manejo de threads
- Aplicar OOP utilizando ideas de _multithreading_ y concurrencia

## Introducción

Los sistemas operativos modernos tienen muchos procesos corriendo en paralelo, donde nuestro programa en Python es solo uno de esos tantos procesos. Podemos obervar este comportamiento incluso en computadores con un sólo procesador, ¿cómo es posible? En realidad, el paralelismo que observamos es solo una ilusión. Lo que hace el sistema operativo es **compartir la CPU entre todos los procesos**, que consiste en tomar un proceso, hacerlo avanzar unas cuantas operaciones, dejarlo en pausa, y volver a repetir lo mismo con otro proceso. Esto ocurre tan rápido que parece que todos los procesos avanzan al mismo tiempo.

En la actualidad, la mayoría de los computadores tiene un procesador con más de un núcleo, por lo que en realidad si pueden hacer más de una operación simultánea. No obstante, la cantidad de procesos es mucho más grande que la de núcleos, por lo que el sistema de igual forma tiene que compartir las CPUs entre los procesos.

En esta unidad, aprenderemos a hacer correr dos o más tareas – dentro de nuestro programa – como si fueran simultáneas. Para ello, usamos _**threads**_.

## Threading

Los **_threads_** (o hilos) son las unidades más pequeñas que pueden ser programadas para ser ejecutadas en un sistema operativo. Son una especie de proceso liviano, que están dentro de un programa o de un proceso mayor. Los _threads_ que están dentro de un mismo proceso comparten la memoria y el estado de las variables. Este uso compartido de los recursos permite a los _threads_ ejecutarse más rápido que correr varias instancias del mismo programa.

Un proceso puede tener uno o varios _threads_. En el momento en que se crea un proceso se crea un _thread_ con él, que llamamos _thread_ principal. Un _thread_ puede crear otros _threads_, los que serán ejecutados **como si fueran tareas en paralelo**.

Como dijimos antes, este paralelismo no es real. Para simular el paralelismo se realiza _thread scheduling_ o _time slicing_, que funciona – en general – turnando los _threads_ de la siguiente manera:

- Se toma un _thread_ de los que están en espera.
- Se ejecuta un cierto número de instrucciones de ese _thread_.
- Se deja el _thread_ en espera.
- Se repite lo mismo con otro _thread_ que está en espera.

Cuando hay un sólo procesador, no existe un verdadero aumento de la velocidad de ejecución, pero sí de la manera en que el programa responde. En máquinas con más procesadores, efectivamente se logra la ejecución en paralelo de varios _threads_, que tiene como resultado una ejecución más rápida del programa. La siguiente figura muestra un esquema de la interacción de los _threads_ con el proceso principal.

![thread](imgs/thread.png)

Algunos ejemplos donde resulta útil implementar _threads_ son:

- Interfaces de rápida respuesta, donde se necesita interactuar con el usuario mientras se ejecuta algún proceso de cómputo pesado. Por ejemplo, las interfaces gráficas en un computador.
- Delegación de trabajos que siguen el patrón _consumidor-productor_, en donde existen procesos que se ejecutan secuencialmente, pero que son independientes entre ellos. Como por ejemplo, un _thread_ que se encargue de poner los _frames_ capturados desde una cámara de video en una cola, y otro _thread_ que procese estos cuadros y los saque de la cola.
- Aplicaciones multiusuario, en donde cada _thread_ se encargaría de las peticiones de cada usuario independientemente. Por ejemplo, un servidor de páginas web debe atender a varios clientes a la vez.

### Creación de _threads_

En Python 3, el manejo de _threads_ se realiza mediante la librería `threading`, en donde los _threads_ son representados por objetos. Muchas de las funcionalidades necesarias están implementadas en esta librería. 

La clase `Thread` representa un hilo. Para crear un _thread_, se requiere especificar la función que ejecutará ese _thread_ a través del parámetro `target`. Para ejecutar el _thread_, se debe llamar al método `start()`. La clase `Thread` también admite el argumento `name`, que es el nombre que le queremos dar al hilo. 

En el siguiente ejemplo, crearemos tres _threads_ `t1`, `w1`, y `w2`, que ejecutarán distintas instancias de las funciones `service` y `worker`.

In [5]:
import threading
import time


def worker():
    current_thread = threading.current_thread()
    print(f"{current_thread.name} partiendo...")
    # Pondremos a dormir el thread por 2 segundos simulando 
    # que ocurre algun proceso dentro de la función
    time.sleep(2) 
    print(f"{current_thread.name} terminando...")

def service():
    current_thread = threading.current_thread()
    print(f"{current_thread.name} partiendo...")
    # Ponemos a dormir el thread por 4 segundos simulando
    # un proceso dentro de la función
    time.sleep(4) 
    print(f"{current_thread.name} terminando...")


# Creamos los threads usando la clase Thread
t1 = threading.Thread(name="Thread 1", target=service)
w1 = threading.Thread(name="Thread 2", target=worker)
w2 = threading.Thread(target=worker)  # En este caso usa el nombre asignado por defecto

# Se inicializan los threads creados
w1.start()
w2.start()
t1.start()

# Todas estas líneas serán ejecutadas mientras los threads se ejecutan independientemente 
# del programa principal
print("Fueron creados 3 threads")
for i in range(10):
    print(i)

Thread 1 terminando...
Thread 2 partiendo...
Thread-10 partiendo...
Thread 1 partiendo...Fueron creados 3 threads
0
1
2
3
4
5
6
7

8
9
Thread 2 terminando...
Thread-10 terminando...
Thread 1 terminando...


En el ejemplo, vemos que una vez inicializados los _threads_, el programa principal continúa con el resto de instrucciones mientras los _threads_ se ejecutan. Luego los tres _threads_ terminan independientemente en distintos momentos. El programa principal esperará a que todos los _threads_ hayan terminado correctamente.

También es posible, mediante el atributo `args` o `kwargs`, pasar los argumentos a la función definida como `target` del _thread_.

In [7]:
import threading
import time


def worker(tiempo):
    current_thread = threading.current_thread()
    print(f"{current_thread.name} partiendo con tiempo={tiempo}")
    time.sleep(tiempo)
    print(f"{current_thread.name} terminando...")

    
# Se crean los threads usando la clase Thread, asociada a la función objetivo para 
# ser ejecutada por el thread, y los atributos de la función son ingresados con 
# args o kwargs

w1 = threading.Thread(name="Worker 1", target=worker, args=(3,))
w2 = threading.Thread(name="Worker 2", target=worker, kwargs={"tiempo": 5})
w1.start()
w2.start()

Worker 1 partiendo con tiempo=3Worker 2 partiendo con tiempo=5

Worker 1 terminando...
Worker 2 terminando...


En concordancia con el modelamiento basado en OOP, podemos crear también un _thread_ heredando desde la clase `Thread` y redefiniendo el método `run()`, el cual se ejecuta luego de hacer `Thread.start()`.

In [2]:
import threading
import time


class Worker(threading.Thread):
    """Este será nuestro nuevo Worker basado en Thread"""
    def __init__(self, tiempo):
        # En el caso de los threads, lo primero es invocar al init original.
        super().__init__()
        self.tiempo = tiempo
    
    def run(self):
        # Este metodo ejecuta el proceso de este thread
        # cuando lo iniciamos mediante el metodo start()
        print(f"{self.name} partiendo...")
        tiempo_partida = time.time()
        time.sleep(self.tiempo) 
        print(f"{self.name} terminando después de {time.time() - tiempo_partida} seg.")

        
class Service(threading.Thread):
    """Este sera el nuevo Service basado en Thread"""
    def __init__(self, tiempo):
        super().__init__()
        self.tiempo = tiempo
    
    def run(self):
        print(f"{self.name} partiendo...")
        tiempo_partida = time.time()
        time.sleep(self.tiempo) 
        print(f"{self.name} terminando después de {time.time() - tiempo_partida} seg.")
        

# Se crean los threads
t1 = Service(3)
t1.name = "Servicio 3"
w1 = Worker(4)
w1.name = "Trabajador 4"
w2 = Worker(5)
w2.name = "Trabajador 5"

# Se inicializan los threads creados
w1.start()
w2.start()
t1.start()

# Todas estas líneas serán ejecutadas mientras los threads se ejecutan independientemente
# del programa principal.
print("Fueron creados 3 threads")
for i in range(10):
    print(i)

Trabajador 4 partiendo...
Trabajador 5 partiendo...
Servicio 3 partiendo...Fueron creados 3 threads
0
1
2
3
4
5
6
7
8
9

Servicio 3 terminando después de 3.001357078552246 seg.
Trabajador 4 terminando después de 4.0005552768707275 seg.
Trabajador 5 terminando después de 5.000891208648682 seg.


### `join()`

Si necesitamos que el programa principal espere a que termine la ejecución de algún _thread_ (o grupo de _threads_), debemos usar del método `join(timeout=None)` después de que el _thread_ ha sido iniciado mediante `start()`.

Al usar `join()`, el programa principal queda bloqueado hasta que los _threads_ referenciados terminen correctamente. No obstante, si se especifica un `timeout` (en segundos) el programa esperará al _thread_ referenciado solo por ese tiempo. 

La figura siguiente explica gráficamente la ejecución del programa usando `join()`.

![join](imgs/join.png)

Veamos ahora el mismo ejemplo anterior pero con la incorporación de `join()`.

In [3]:
import threading
import time

# Usamos la definicion de los Thread declarados en el ejemplo anterior
# Se crean los threads usando la clase Thread.
t1 = Service(1)
w1 = Worker(6)
w2 = Worker(10)

# Se inicializan los threads creados
t1.start()
w1.start()
w2.start()

# Aquí incorporamos el método join() para bloquear el programa principal
t1.join()     # No especificamos timeout, esperará lo que sea necesario
w1.join()     # Esperaremos lo que sea necesario.
w2.join(3)  # Esperaremos máximo 8.8 segundos.

# En este punto, el programa ha esperado por los tres threads que creamos
# Estas líneas serán ejecutadas después de que los threads hayan terminado

print("Fueron creados 3 threads")
for i in [t1,w1,w2]:
    print(i.is_alive())

Thread-9 partiendo...
Thread-10 partiendo...
Thread-11 partiendo...
Thread-9 terminando después de 1.0016248226165771 seg.
Thread-10 terminando después de 6.001014471054077 seg.
Fueron creados 3 threads
False
False
True
Thread-11 terminando después de 10.001221656799316 seg.


### `is_alive()`

Es posible identificar si un _thread_ todavía está en funcionamiento mediante el uso del método `is_alive()`. Por lo general este método se implementa para saber el estado del _thread_ después del uso de `join()`, al cual se le ha definido su tiempo máximo de espera. Usando `Service` y `Worker` podemos hacer lo siguiente:

In [4]:
import threading
import time

# Se crean los threads usando la clase Thread
t1 = Service(2)
w1 = Worker(10)

# Se inicializan los threads
t1.start()
w1.start()

# Esperaremos a que t1 termine, por 2 segundos
t1.join(2)
# Esperaremos a que w1 termine, por 2 segundos más
w1.join(2)

# Dado como definimos Service y Worker, es muy probable que Service
# haya terminado en los 2 segundos, pero poco probable que 
# Worker haya terminado.

# Podemos comprobar si los threads están activos o no

for thread in [t1, w1]:
    if thread.is_alive():
        print(f"El thread {thread.name} sigue activo :(")
    else:
        print(f"El thread {thread.name} ha terminado :)")

Thread-12 partiendo...
Thread-13 partiendo...
Thread-12 terminando después de 2.0005407333374023 seg.
El thread Thread-12 ha terminado :)
El thread Thread-13 sigue activo :(
Thread-13 terminando después de 10.000402212142944 seg.


### _Daemons_

En los ejemplos anteriores, el programa espera a que todos los _threads_ terminen su trabajo antes de terminar su ejecución. Los **_daemon threads_** son los que, a pesar de que estén aún corriendo, no impiden que el programa principal termine.

Si no existieran los _daemon threads_, tendríamos que preocuparnos de tener en cuenta todos nuestros _threads_ y preocuparnos de terminarlos a todos para que el programa principal pueda terminar. Con los _daemon threads_, podemos ponerlos a correr y olvidarnos de ellos, y cuando el programa termina, los _daemon threads_ serán terminados inmediatamente. El programa principal terminará cuando todos los _threads no-daemon_ hayan finalizado.

Para especificar que un _thread_ debe ser _daemon_, se especifica el parámetro `daemon` en el constructor del _thread_. Una forma alternativa es escribir el atributo `daemon` del _thread_, después de haberlo creado.

In [5]:
import threading
import time


def worker():
    print(f"{threading.current_thread().name} partiendo...")
    time.sleep(2)
    print(f"{threading.current_thread().name} saliendo...")

    
def service():
    print(f"{threading.current_thread().name} partiendo...")
    time.sleep(4)
    print(f"{threading.current_thread().name} saliendo...")


# Forma 1 de hacer un thread daemon
t1 = threading.Thread(name="Service", target=service, daemon=True)
# Forma 2 de hacer un thread daemon
w1 = threading.Thread(name="Worker", target=worker)
w1.daemon = True

# Se inicializan los threads
w1.start()
t1.start()

Worker partiendo...
Service partiendo...
Worker saliendo...
Service saliendo...


La siguiente figura muestra la salida en la consola para el programa **con** los _threads_ configurados como `Daemon`. Los _threads_ se interrumpen abruptamente, luego de que el programa principal termina su ejecución.

![daemon](imgs/daemon.png)

Comparemos el resultado anterior con la salida en la consola para el programa **sin** los _threads_ configurados como `Daemon`, como muestra la imagen siguiente. Podemos ver que la ejecución de los _threads_ fue completada y que el programa no se cierra hasta que los _threads_ terminan.

![non_daemon](imgs/non-daemon.png)

En la celda anterior, podemos apreciar que el programa principal termina antes que el _daemon thread_. Es posible hacer que el programa espere a un _daemon thread_ mediante el uso de `join()`, lo que tendría sentido cuando debemos esperar algún resultado del _thread_ para el resto del programa.

In [6]:
import threading
import time


def worker():
    print(f"{threading.current_thread().name} partiendo...")
    time.sleep(2)
    print(f"{threading.current_thread().name} saliendo...")

    
def service():
    print(f"{threading.current_thread().name} partiendo...")
    time.sleep(4)
    print(f"{threading.current_thread().name} saliendo...")


# Forma 1 de hacer un thread daemon
t1 = threading.Thread(name="Service", target=service, daemon=True)
# Forma 2 de hacer un thread daemon
w1 = threading.Thread(name="Worker", target=worker)
w1.daemon = True

# Se inicializan los threads
w1.start()
t1.start()

# Esperamos los threads.
# Lo esperamos por una cantidad indefinida de tiempo
w1.join()
# Esperamos sólo 5 segundos
w2.join(5)

Worker partiendo...
Service partiendo...
Worker saliendo...
Service saliendo...


Una cosa importante a notar es que una vez que se ha llamado el método `start()` un _thread_ no puede cambiar desde _daemon_ a _no-daemon_, o vicecersa. Si tratamos de hacerlo, se lanza una excepción del tipo `RuntimeError`.

In [7]:
t1 = threading.Thread(name="Service", target=service, daemon=True)
t1.start()

# Noo!
t1.daemon = False

t1.join()

Service partiendo...


RuntimeError: cannot set daemon status of active thread

Service saliendo...


Al hacer una subclase de `Thread`, que queremos que sea _daemon_, podemos indicar que el _thread_ es _daemon_ durante su inicialización, usando el atributo `daemon`.

In [8]:
class Daemon(threading.Thread):
    
    def __init__(self):
        super().__init__()
        # Cuando inicializamos el thread lo declaramos como daemon
        self.daemon = True
    
    def run(self):
        print("Empezando...")
        time.sleep(2)
        print("Terminando...")

daemon = Daemon()
daemon.start()
daemon.join()

Empezando...
Terminando...


### _Timers_

La clase `Timer` es una subclase de la clase `Thread` y permite ejecutar un proceso o acción después que ha pasado un determinado tiempo. `Timer` requiere, como parámetros básicos, el tiempo en segundos después del cual debe ejecutarse el _thread_, la función a ejecutar, y los argumentos de la función a ejecutar.

El método `cancel()` permite, si se requiere, cancelar la ejecución del _timer_ antes que este sea ejecuctado.

In [9]:
def mi_timer(archivo):
    with open(archivo) as fid:
        for linea in fid:
            print(linea)

t1 = threading.Timer(10.0, mi_timer, args=("files/mensaje_01.txt",))
t2 = threading.Timer(5.0, mi_timer, kwargs={"archivo": "files/mensaje_02.txt"})

t1.start() # el thread t comenzará después de 10 seconds
t2.start() # el thread t comenzará después de 5 seconds

Hola :) acabas de ejecutar el timer 2

Hola! acabas de ejecutar el timer 1
